In [ ]:
'''
PPCA 3.0 xxxx

xxxx

Requirements:
- A specific working environment (see github page of the project)
- Geopackage file from PPCA 2.0

Guide to run the script:
- Fill 0.2 box

Author : Perez, Joan 
'''

In [1]:
# 0.1 : libraries
import pandas as pd
import geopandas as gpd
import numpy as np
import momepy
import libpysal
import warnings

In [2]:
# 0.2 : Box to fil with informations

# Name of the case study
Name = 'Nice'

# Recall the ghs date (extracted in step 1.0)
ghs_date = 2020

# Define projected CRS
projected_crs = 'EPSG:2154'

In [3]:
gpkg = f'PPCA1_{Name}.gpkg'
gpkg2 = f'PPCA2_{Name}.gpkg'
building = gpd.read_file(gpkg, layer = 'osm_all_buildings')
ghs_vector = gpd.read_file(gpkg2, layer = f'ghs_populated_{ghs_date}_vector')

In [4]:
# Ensure height & level columns are numeric
building['height'] = pd.to_numeric(building['height'], errors='coerce')
building['building:levels'] = pd.to_numeric(building['building:levels'], errors='coerce')

# checks if height is NaN and if floor is not NaN.
# If both conditions are met : multiplies the value of floor by 3 and assigns it to height
building['height'] = building.apply(lambda row: row['building:levels'] * 3 if pd.isna(row['height'])
                                     and not pd.isna(row['building:levels']) else row['height'], axis=1)
# checks if the height column is NaN and if the floor is not NaN.
# If both conditions are met : multiplies the value of floor by 3 and assigns it to height
building['building:levels'] = building.apply(lambda row: round(row['height'] / 3) if pd.isna(row['building:levels'])
                                             and not pd.isna(row['height']) else row['building:levels'], axis=1)

# Calculate the number and percentage of rows with NaN for both 'height' and 'building:levels'
num_na = building[['height', 'building:levels']].isna().all(axis=1).sum()
percent_na = (num_na / len(building)) * 100

# Print the number and percentage of rows with NA for both columns
print(f'{Name} : Number of remaining values with NA for height/floors: {num_na} ({percent_na:.2f}%)')

Nice : Number of remaining values with NA for height/floors: 9473 (29.17%)


In [5]:
building = building.to_crs(projected_crs)

# Number of floors
building.rename(columns={'building:levels': 'FL'}, inplace=True)

# Surface
building['A'] = building.geometry.area

# Calculating perimeter
building['P'] = building.geometry.length

# Calculating elongation
building['E'] = momepy.Elongation(building).series

# Convexity
building['C'] = momepy.Convexity(building).series

# Floor area
building['FA'] = building['FL'] * building['A']

# Product [1-E].C.S
building['ECA'] = (1 - building['E']) * building['A'] * building['C']

# [1-E].S
building['EA'] = (1 - building['E']) * building['A']

# Shared walls
building["SW"] = momepy.SharedWallsRatio(building).series

C:\Users\jperez\anaconda3\envs\PPCA3\Lib\site-packages\momepy\distribution.py:137: FutureWarning: The `query_bulk()` method is deprecated and will be removed in GeoPandas 1.0. You can use the `query()` method instead.
  inp, res = gdf.sindex.query_bulk(gdf.geometry, predicate="intersects")
